In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
from playwright.async_api import Page
from typing import Optional, Dict, Any

class AccessibilityMapper:
    def __init__(self, page: Page):
        self.page = page
        self.cdp_client = None

    async def init_cdp(self):
        """Initialize CDP client if not already done"""
        if not self.cdp_client:
            self.cdp_client = await self.page.context.new_cdp_session(self.page)

    async def get_accessibility_tree(self) -> Dict[str, Any]:
        """Get the full accessibility tree using CDP"""
        await self.init_cdp()
        return await self.cdp_client.send("Accessibility.getFullAXTree")

    async def get_dom_element(self, ax_node_id: str):
        """Get DOM element from accessibility node ID"""
        await self.init_cdp()
        # First get the backend DOM node ID
        node_info = await self.cdp_client.send("Accessibility.getBackendNodeIdForAXNode", {
            "id": ax_node_id
        })
        
        # Then get a DOM element handle using the backend node ID
        element = await self.page.evaluate_handle(f"""(backend_id) => {{
            const nodes = document.evaluate(
                '//*',
                document,
                null,
                XPathResult.ORDERED_NODE_SNAPSHOT_TYPE,
                null
            );
            for (let i = 0; i < nodes.snapshotLength; i++) {{
                const node = nodes.snapshotItem(i);
                if (node.__playwright_backend_node_id === {node_info['backendDOMNodeId']}) {{
                    return node;
                }}
            }}
            return null;
        }}""")
        
        return element


In [5]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
# initialize your page first
browser = await playwright.chromium.launch()
page = await browser.new_page()

# Create mapper
mapper = AccessibilityMapper(page)

# Get accessibility snapshot
snapshot = await page.accessibility.snapshot()

# Get the full CDP accessibility tree
ax_tree = await mapper.get_accessibility_tree()

# For any node in the tree, you can get its DOM element
for node in ax_tree['nodes']:
    element = await mapper.get_dom_element(node['nodeId'])
    if element:
        # Now you can interact with the element
        await element.click()
        # Or get its properties
        props = await element.get_properties()

Error: CDPSession.send: Protocol error (Accessibility.getBackendNodeIdForAXNode): 'Accessibility.getBackendNodeIdForAXNode' wasn't found